In [1]:
import os
from dotenv import load_dotenv
import wrds
import pandas as pd

load_dotenv()
PGUSER = os.getenv('PGUSER')
PGPASSWORD = os.getenv('PGPASSWORD')

In [2]:
conn = wrds.Connection()

Loading library list...
Done


In [22]:
# conn.get_table(library='crsp', table='dsp500list', rows=10)
# conn.get_table(library='crsp', table='dsf', rows=10)

# getting DAILY returns for SP500 constituents + date range of their belonging to SP500
df = conn.raw_sql(
    """select a.*, b.date, b.ret
        from crsp.dsp500list as a,
        crsp.dsf as b
        where a.permno = b.permno
        and b.date >= a.start and b.date <= a.ending
        and b.date >= '01/01/2003'""", date_cols=['start', 'ending', 'date']
    )

# getting additional company attributes / identifiers
temp = conn.raw_sql(
    """select comnam, namedt, nameendt, permno, shrcd, exchcd, ticker
        from crsp.dsenames""", date_cols=['namedt', 'nameendt']
    )

# merging, subsetting
df = df.merge(temp, how='inner', on='permno')
temp = None

# return date is between the range for which company name was used
df = df.loc[(df.date >= df.namedt) & (df.date <= df.nameendt)]

# for exchcd (exchange code) we have 1: NYSE, 2: NYSE MKT, 3: NASDAQ, 4: ARCA
# for shrcd (share code) we have 11: ordinary common shares, need no further definition, US incorporated
df = df.loc[(df.exchcd.isin((1, 2, 3, 4))) & (df.shrcd == 11)]

In [31]:
# permno: permanent code for security (guaranteed to not change versus something like ticker)
# start, ending: start and end dates of SP500 membership
# date, ret: daily return for stock on given date
df.head(5)

,permno,start,ending,date,ret,comnam,namedt,nameendt,shrcd,exchcd,ticker
0,10078,1992-08-20,2010-01-28,2003-01-02,0.086817,SUN MICROSYSTEMS INC,1986-03-04,2004-06-09,11,3,SUNW
6,10104,1989-08-03,2023-12-29,2003-01-02,0.037963,ORACLE CORP,1995-06-01,2004-06-09,11,3,ORCL
11,10104,1989-08-03,2023-12-29,2003-01-03,0.031222,ORACLE CORP,1995-06-01,2004-06-09,11,3,ORCL
16,10104,1989-08-03,2023-12-29,2003-01-06,0.034602,ORACLE CORP,1995-06-01,2004-06-09,11,3,ORCL
21,10104,1989-08-03,2023-12-29,2003-01-07,0.061037,ORACLE CORP,1995-06-01,2004-06-09,11,3,ORCL


In [35]:
# caldt  := calendar date
# vwretd := value-weighted market portfolio return
# sprtrn := SP500 return
conn.get_table(library='crsp', table='dsp500', columns=['caldt', 'vwretd', 'sprtrn'], date_cols=['caldt'])

,caldt,vwretd,sprtrn
0,1925-12-31,None,None
1,1926-01-02,0.004297,None
2,1926-01-04,-0.001357,None
3,1926-01-05,-0.004603,None
4,1926-01-06,0.000537,None
...,...,...,...
25794,2023-12-22,0.001708,0.001660
25795,2023-12-26,0.004208,0.004232
25796,2023-12-27,0.001604,0.001430
25797,2023-12-28,0.000415,0.000370


In [ ]:
# conn.close()